In [29]:
import numpy as np


class DmipyAcquisitionScheme:
    """
    Class that calculates and contains all information needed to simulate and
    fit data using microstructure models.
    """

    def __init__(self, bvalues, gradient_directions, qvalues, gradient_strengths, delta, Delta, TE, TI,
                 min_b_shell_distance, b0_threshold):
        
        self.min_b_shell_distance = float(min_b_shell_distance)
        self.b0_threshold = float(b0_threshold)
        self.bvalues = bvalues.astype(float)
        self.b0_mask = self.bvalues <= b0_threshold
        self.number_of_b0s = np.sum(self.b0_mask)
        self.number_of_measurements = len(self.bvalues)
        self.gradient_directions = gradient_directions.astype(float)
        
        self.qvalues = None
        if qvalues is not None:
            self.qvalues = qvalues.astype(float)
        
        self.gradient_strengths = None
        if gradient_strengths is not None:
            self.gradient_strengths = gradient_strengths.astype(float)
        
        self.delta = None
        if delta is not None:
            self.delta = delta.astype(float)
        
        self.Delta = None
        if Delta is not None:
            self.Delta = Delta.astype(float)
        
        self.TE = None
        self.N_TE = 1  # default if not given
        if TE is not None:
            self.TE = TE.astype(float)
            
        self.TI = None
        self.N_TI = 1  # default if not given
        if TI is not None:
            self.TI = TI.astype(float)
        
        self.tau = None
        if self.delta is not None and self.Delta is not None:
            self.tau = Delta - delta / 3.
        
        # if there are more then 1 measurement
        if self.number_of_measurements > 1:
            # we check if there are multiple unique delta-Delta combinations
            if self.TE is not None:
                deltas = np.c_[self.delta, self.Delta, self.TE]
            elif self.delta is not None and self.Delta is not None:
                deltas = np.c_[self.delta, self.Delta]
            elif self.delta is None and self.Delta is not None:
                deltas = np.c_[self.Delta]
            elif self.delta is not None and self.Delta is None:
                deltas = np.c_[self.delta]
            else:
                deltas = []

            if deltas == []:
                deltas = np.c_[np.zeros(len(self.bvalues))]
            unique_deltas = np.unique(deltas, axis=0)
            self.shell_indices = np.zeros(len(bvalues), dtype=int)
            self.shell_bvalues = []
            max_index = 0
            # for every unique combination we separate shells based on bvalue
            # reason for separation is that different combinations of
            # delta and Delta can result in the same b-value, which could
            # result in wrong classification of DWIs to unique shells.
            for unique_deltas_ in unique_deltas:
                delta_mask = np.all(deltas == unique_deltas_, axis=1)
                masked_bvals = bvalues[delta_mask]
                if len(masked_bvals) > 1:
                    shell_indices_, shell_bvalues_ = (
                        calculate_shell_bvalues_and_indices(masked_bvals, min_b_shell_distance)
                    )
                else:
                    shell_indices_, shell_bvalues_ = np.array(0), masked_bvals
                self.shell_indices[delta_mask] = shell_indices_ + max_index
                self.shell_bvalues.append(shell_bvalues_)
                max_index = max(self.shell_indices + 1)
            self.shell_bvalues = np.hstack(self.shell_bvalues)
            self.shell_b0_mask = self.shell_bvalues <= b0_threshold

            first_indices = [
                np.argmax(self.shell_indices == ind)
                for ind in np.arange(self.shell_indices.max() + 1)]
            self.shell_qvalues = None
            if self.qvalues is not None:
                self.shell_qvalues = self.qvalues[first_indices]
            self.shell_gradient_strengths = None
            if self.gradient_strengths is not None:
                self.shell_gradient_strengths = (
                    self.gradient_strengths[first_indices])
            self.shell_delta = None
            if self.delta is not None:
                self.shell_delta = self.delta[first_indices]
            self.shell_Delta = None
            if self.Delta is not None:
                self.shell_Delta = self.Delta[first_indices]
            self.shell_TE = None
            if self.TE is not None:
                self.shell_TE = self.TE[first_indices]
                if (len(np.unique(self.TE)) != len(np.unique(
                        self.TE[self.b0_mask]))):
                    msg = "Not every TE shell has b0 measurements.\n"
                    msg += "This is required to properly normalize the signal."
                    msg += " Make sure the TE values for b0-measurements have "
                    msg += "not defaulted to 0 for example."
                    raise ValueError(msg)
                self.N_TE = len(self.shell_TE)
            if self.TI is not None:
                self.shell_TI = self.TI[first_indices]
                if (len(np.unique(self.TI)) != len(np.unique(
                        self.TI[self.b0_mask]))):
                    msg = "Not every TI shell has b0 measurements.\n"
                    msg += "This is required to properly normalize the signal."
                    msg += " Make sure the TI values for b0-measurements have "
                    msg += "not defaulted to 0 for example."
                    raise ValueError(msg)
                self.N_TI = len(self.shell_TI)    
            
        # if for some reason only one measurement is given (for testing)
        else:
            self.shell_bvalues = self.bvalues
            self.shell_indices = np.r_[int(0)]
            if self.shell_bvalues > b0_threshold:
                self.shell_b0_mask = np.r_[False]
            else:
                self.shell_b0_mask = np.r_[True]
            self.shell_qvalues = self.qvalues
            self.shell_gradient_strengths = self.gradient_strengths
            self.shell_delta = self.delta
            self.shell_Delta = self.Delta
            self.shell_TE = TE
            self.shell_TI = TI

        # calculates observation matrices to convert spherical harmonic
        # coefficients to the positions on the sphere for every shell
        self.unique_b0_indices = np.unique(self.shell_indices[self.b0_mask])
        self.unique_dwi_indices = np.unique(self.shell_indices[~self.b0_mask])
        self.unique_shell_indices = np.unique(self.shell_indices)
        self.N_b0_shells = len(self.unique_b0_indices)
        self.N_dwi_shells = len(self.unique_dwi_indices)
        self.N_shells = len(self.unique_shell_indices)
        self.shell_sh_matrices = {}
        self.shell_sh_orders = {}
        for shell_index in self.unique_b0_indices:
            self.shell_sh_orders[shell_index] = 0
        for shell_index in self.unique_dwi_indices:
            shell_mask = self.shell_indices == shell_index
            bvecs_shell = self.gradient_directions[shell_mask]
            _, theta_, phi_ = cart2sphere(bvecs_shell).T
            self.shell_sh_orders[shell_index] = get_sh_order_from_bval(
                self.shell_bvalues[shell_index])
            self.shell_sh_matrices[shell_index] = real_sym_sh_mrtrix(
                self.shell_sh_orders[shell_index], theta_, phi_)[0]
        # warning in case there are no b0 measurements
        if sum(self.b0_mask) == 0:
            msg = "No b0 measurements were detected. Check if the b0_threshold"
            msg += " option is high enough, or if there is a mistake in the "
            msg += "acquisition design."
            warn(msg)


    @property
    def print_acquisition_info(self):
        """
        prints a small summary of the acquisition scheme. Is useful to check if
        the function correctly separated the shells and if the input parameters
        were given in the right scale.
        """
        print("Acquisition scheme summary\n")
        print("total number of measurements: {}".format(
            self.number_of_measurements))
        print("number of b0 measurements: {}".format(self.number_of_b0s))
        print("number of DWI shells: {}\n".format(
            np.sum(~self.shell_b0_mask)))
        upper_line = "shell_index |# of DWIs |bvalue [s/mm^2] "
        upper_line += "|gradient strength [mT/m] |delta [ms] |Delta[ms]"
        upper_line += " |TE[ms] |TI[ms]"
        print(upper_line)
        for ind in np.arange(max(self.shell_indices) + 1):
            if (self.shell_TI is not None and self.shell_TE is not None and 
                self.shell_delta is not None and self.shell_Delta is not None):
                print(
                    "{:<12}|{:<10}|{:<16}|{:<25}|{:<11}|{:<10}|{:<5}|{:<5}".format(
                        str(ind), sum(self.shell_indices == ind),
                        int(self.shell_bvalues[ind] / 1e6),
                        int(1e3 * self.shell_gradient_strengths[ind]),
                        self.shell_delta[ind] * 1e3, self.shell_Delta[ind] * 1e3, 
                        self.shell_TE[ind] * 1e3, self.shell_TI[ind] * 1e3))




In [30]:
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage
from dipy.reconst.shm import real_sym_sh_mrtrix


def calculate_shell_bvalues_and_indices(bvalues, max_distance=20e6):
    """
    Calculates which measurements belong to different acquisition shells.
    It uses scipy's linkage clustering algorithm, which uses the max_distance
    input as a limit of including measurements in the same cluster.
    For example, if bvalues were [1, 2, 3, 4, 5] and max_distance was 1, then
    all bvalues would belong to the same cluster.
    However, if bvalues were [1, 2, 4, 5] max max_distance was 1, then this
    would result in 2 clusters.
    Parameters
    ----------
    bvalues: 1D numpy array of shape (Ndata)
        bvalues of the acquisition in s/m^2.
    max_distance: float
        maximum b-value distance for a measurement to be included in the same
        shell.
    Returns
    -------
    shell_indices: 1D numpy array of shape (Ndata)
        array of integers, starting from 0, representing to which shell a
        measurement belongs. The number itself has no meaning other than just
        being different for different shells.
    shell_bvalues: 1D numpy array of shape (Nshells)
        array of the mean bvalues for every acquisition shell.
    """
    linkage_matrix = linkage(np.c_[bvalues])
    clusters = fcluster(linkage_matrix, max_distance, criterion='distance')
    shell_indices = np.empty_like(bvalues, dtype=int)
    cluster_bvalues = np.zeros((np.max(clusters), 2))
    for ind in np.unique(clusters):
        cluster_bvalues[ind - 1] = np.mean(bvalues[clusters == ind]), ind
    shell_bvalues, ordered_cluster_indices = (
        cluster_bvalues[cluster_bvalues[:, 0].argsort()].T)
    for i, ind in enumerate(ordered_cluster_indices):
        shell_indices[clusters == ind] = i
    return shell_indices, shell_bvalues


def get_sh_order_from_bval(bval):
    "Estimates minimum sh_order to represent data of given b-value."
    bvals = np.r_[2.02020202e+08, 7.07070707e+08, 1.21212121e+09,
                  2.52525253e+09, 3.13131313e+09, 5.35353535e+09,
                  np.inf]
    sh_orders = np.arange(2, 15, 2)
    return sh_orders[np.argmax(bvals > bval)]


def cart2sphere(cartesian_coordinates):
    """"Function to estimate spherical coordinates from cartesian coordinates
    according to wikipedia. Conforms with the dipy notation.
    Parameters
    ----------
    cartesian_coordinates : array of size (3) or (N x 3),
        array of cartesian coordinate vectors [x, y, z].
    Returns
    -------
    spherical_coordinates : array of size (3) or (N x 3),
        array of spherical coordinate vectors [r, theta, phi].
        range of theta [0, pi]. range of phi [-pi, pi].
    """
    if np.ndim(cartesian_coordinates) == 1:
        x, y, z = cartesian_coordinates
        r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
        if r > 0:
            theta = np.arccos(z / r)
        else:
            theta = 0
        phi = np.arctan2(y, x)
        spherical_coordinates = np.r_[r, theta, phi]
    elif np.ndim(cartesian_coordinates) == 2:
        x, y, z = cartesian_coordinates.T
        r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
        theta = np.arccos(z / r)
        theta = np.where(r > 0, theta, 0.)
        phi = np.arctan2(y, x)
        spherical_coordinates = np.c_[r, theta, phi]
    else:
        msg = "coordinates must be array of size 3 or N x 3."
        raise ValueError(msg)
    return spherical_coordinates

In [31]:
def acquisition_scheme_from_bvalues(bvalues, gradient_directions, delta=None, Delta=None, TE=None, TI=None,
                                    min_b_shell_distance=50e6, b0_threshold=10e6):
    r"""
    Creates an acquisition scheme object from bvalues, gradient directions,
    pulse duration $\delta$ and pulse separation time $\Delta$.
    Parameters
    ----------
    bvalues: 1D numpy array of shape (Ndata)
        bvalues of the acquisition in s/m^2.
        e.g., a bvalue of 1000 s/mm^2 must be entered as 1000 * 1e6 s/m^2
    gradient_directions: 2D numpy array of shape (Ndata, 3)
        gradient directions array of cartesian unit vectors.
    delta: float or 1D numpy array of shape (Ndata)
        if float, pulse duration of every measurements in seconds.
        if array, potentially varying pulse duration per measurement.
    Delta: float or 1D numpy array of shape (Ndata)
        if float, pulse separation time of every measurements in seconds.
        if array, potentially varying pulse separation time per measurement.
    min_b_shell_distance : float
        minimum bvalue distance between different shells. This parameter is
        used to separate measurements into different shells, which is necessary
        for any model using spherical convolution or spherical mean.
    b0_threshold : float
        bvalue threshold for a measurement to be considered a b0 measurement.
    Returns
    -------
    DmipyAcquisitionScheme: acquisition scheme object
        contains all information of the acquisition scheme to be used in any
        microstructure model.
    """
    delta_, Delta_, TE_, TI_ = unify_length_reference_delta_Delta(
        bvalues, delta, Delta, TE, TI)
    if delta is not None and Delta is not None:
        qvalues = q_from_b(bvalues, delta_, Delta_)
        gradient_strengths = g_from_b(bvalues, delta_, Delta_)
    else:
        qvalues = gradient_strengths = None
    return DmipyAcquisitionScheme(bvalues, gradient_directions, qvalues, gradient_strengths, delta_, Delta_, TE_,
                                  TI_, min_b_shell_distance, b0_threshold)


def unify_length_reference_delta_Delta(reference_array, delta, Delta, TE, TI):
    """
    If either delta or Delta are given as float, makes them an array the same
    size as the reference array.
    Parameters
    ----------
    reference_array : array of size (Nsamples)
        typically b-values, q-values or gradient strengths.
    delta : float or array of size (Nsamples)
        pulse duration in seconds.
    Delta : float or array of size (Nsamples)
        pulse separation in seconds.
    TE : None, float or array of size (Nsamples)
        Echo time of the acquisition in seconds.
    Returns
    -------
    delta_ : array of size (Nsamples)
        pulse duration copied to be same size as reference_array
    Delta_ : array of size (Nsamples)
        pulse separation copied to be same size as reference_array
    TE_ : None or array of size (Nsamples)
        Echo time copied to be same size as reference_array
    """
    if delta is None:
        delta_ = delta
    elif isinstance(delta, float) or isinstance(delta, int):
        delta_ = np.tile(delta, len(reference_array))
    else:
        delta_ = delta.copy()
    if Delta is None:
        Delta_ = Delta
    elif isinstance(Delta, float) or isinstance(Delta, int):
        Delta_ = np.tile(Delta, len(reference_array))
    else:
        Delta_ = Delta.copy()
    if TE is None:
        TE_ = TE
    elif isinstance(TE, float) or isinstance(TE, int):
        TE_ = np.tile(TE, len(reference_array))
    else:
        TE_ = TE.copy()
    if TI is None:
        TI_ = TI
    elif isinstance(TI, float) or isinstance(TI, int):
        TI_ = np.tile(TI, len(reference_array))
    else:
        TI_ = TI.copy()
    return delta_, Delta_, TE_, TI_                


def q_from_b(b, delta, Delta):
    """Compute q-value from b-value. Units are standard units."""
    tau = Delta - delta / 3
    q = np.sqrt(b / tau) / (2 * np.pi)
    return q


CONSTANTS = dict(
    water_diffusion_constant=2.299e-9,  # m^2/s
    water_in_axons_diffusion_constant=1.7e-9,  # m^2/s
    naa_in_axons=.00015e-9,  # m^2 / s
    water_gyromagnetic_ratio=267.513e6,   # 1/(sT)
)


def g_from_b(
    b, delta, Delta,
    gyromagnetic_ratio=CONSTANTS['water_gyromagnetic_ratio']
):
    """Compute gradient strength from b-value. Units are standard units."""
    tau = Delta - delta / 3
    return np.sqrt(b / tau) / (gyromagnetic_ratio * delta)


In [32]:
# Load brain data and mask

# Access nifti, bval & bvec files
from os.path import expanduser, join
home = expanduser('~')
dname = join(home, 'brain-data-neil')

fdwi = join(dname, 'cdmri11_r.nii')
mask = join(dname, 'vol0083_brain_mask.nii.gz')
fbval = join(dname, 'parameters_new_bval.txt')
fbvec = join(dname, 'parameters_new_bvec2.txt')

# Load dMRI datasets 
from dipy.io.image import load_nifti
data, affine, img = load_nifti(fdwi, return_img=True)
mask_data, affine1, img1 = load_nifti(mask, return_img=True)

# Check size of data --> (77, 92, 56, 1344)
print(data.shape)

(77, 92, 56, 1344)


In [33]:
# Set up acquisition scheme using bval, bvec

# Import relevant modules
from os.path import join
import numpy as np

# Load parameters and convert to SI units
bvalues = np.loadtxt(join('parameters_new_bval.txt'))  # given in s/m^2
bvalues_SI = bvalues * 1e6 # now given in SI units as s/mm^2
gradient_directions = np.loadtxt(join('parameters_new_bvec.txt'))
delta = 0.0242 # time in seconds
Delta = 0.0391 # time in seconds
grad_echo_inv = np.loadtxt(join('parameters_new.txt'))
TE = grad_echo_inv[:,5]/1e3
TI = grad_echo_inv[:,4]/1e3

# Acquisition scheme
acq_scheme = acquisition_scheme_from_bvalues(bvalues_SI, gradient_directions, delta, Delta, TE, TI)
acq_scheme.print_acquisition_info

Acquisition scheme summary

total number of measurements: 1344
number of b0 measurements: 84
number of DWI shells: 12

shell_index |# of DWIs |bvalue [s/mm^2] |gradient strength [mT/m] |delta [ms] |Delta[ms] |TE[ms] |TI[ms]
0           |28        |0               |0                        |24.2       |39.1      |80.0 |1059.7
1           |28        |500             |19                       |24.2       |39.1      |80.0 |883.12
2           |84        |1000            |27                       |24.2       |39.1      |80.0 |176.62
3           |140       |2000            |39                       |24.2       |39.1      |80.0 |529.87
4           |168       |3000            |48                       |24.2       |39.1      |80.0 |20.0 
5           |28        |0               |0                        |24.2       |39.1      |105.0|1059.7
6           |28        |500             |19                       |24.2       |39.1      |105.0|883.12
7           |84        |1000            |27             

<ipython-input-29-7c5bc46b37b9>:65: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if deltas == []:
